# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846900852341                   -0.70    4.8         
  2   -7.852323778996       -2.27       -1.53    1.0   16.9ms
  3   -7.852616283927       -3.53       -2.57    1.5   18.8ms
  4   -7.852646045540       -4.53       -2.91    2.5   22.8ms
  5   -7.852646534165       -6.31       -3.23    1.2   17.7ms
  6   -7.852646679580       -6.84       -4.13    1.0   17.1ms
  7   -7.852646686692       -8.15       -5.14    2.2   22.6ms
  8   -7.852646686728      -10.45       -5.68    1.8   20.2ms
  9   -7.852646686729      -12.07       -5.70    1.5   19.3ms
 10   -7.852646686730      -11.91       -6.93    1.0   17.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846908970987                   -0.70           4.8         
  2   -7.852528066161       -2.25       -1.64   0.80    2.0    204ms
  3   -7.852635518608       -3.97       -2.74   0.80    1.0   15.9ms
  4   -7.852646588783       -4.96       -3.23   0.80    2.2   21.2ms
  5   -7.852646681301       -7.03       -4.09   0.80    1.0   16.1ms
  6   -7.852646686282       -8.30       -4.87   0.80    1.2   17.3ms
  7   -7.852646686723       -9.36       -5.66   0.80    2.2   20.6ms
  8   -7.852646686729      -11.18       -6.92   0.80    1.8   19.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.373574e+01     3.320109e+00
 * time: 0.045697927474975586
     1     1.017169e+00     1.674061e+00
 * time: 0.21391010284423828
     2    -1.304440e+00     1.948903e+00
 * time: 0.2317650318145752
     3    -3.616123e+00     1.581499e+00
 * time: 0.2573230266571045
     4    -4.871243e+00     1.497208e+00
 * time: 0.2827339172363281
     5    -6.651794e+00     9.256138e-01
 * time: 0.3081989288330078
     6    -7.298210e+00     5.081583e-01
 * time: 0.33371400833129883
     7    -7.594213e+00     4.037623e-01
 * time: 0.35148191452026367
     8    -7.711240e+00     1.279673e-01
 * time: 0.36914706230163574
     9    -7.758979e+00     1.347486e-01
 * time: 0.38686609268188477
    10    -7.780144e+00     7.444585e-02
 * time: 0.4046039581298828
    11    -7.794406e+00     8.630902e-02
 * time: 0.42233800888061523
    12    -7.809602e+00     8.446130e-02
 * time: 0.4401741027832031
    13    -7.821611e+00     5.777870e-02
 * time: 0.45

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846688456964                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645940340                   -1.64         
  2   -7.852646686730       -6.13       -3.72    1.50s
  3   -7.852646686730      -13.35       -7.27    150ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.9446078980320816e-7
|ρ_newton - ρ_scfv| = 4.2808761609462787e-7
|ρ_newton - ρ_dm|   = 8.371919328577789e-10
